In [1]:
# Install YOLOv8 and Pytorch for using YOLO

from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pydicom
from pydicom.uid import RLELossless
import cv2
# Load a model
model = YOLO('best.pt')  # load a custom trained



In [2]:
# Get dcm files in a folder and return a list
import os
def get_file_list(input_path, extension) :
    try :
        list_path = []
        list_file = []
        list_full = []   
        
        for (path, _, file) in os.walk(input_path):
            for each_file in file:
                if each_file.split('.')[-1] == extension:
                    list_path.append(path)    
                    list_file.append(each_file)
                    list_full.append(os.path.join(os.getcwd(),path,each_file).replace('.\\',''))
        return list_full
    except : 
        return 'get_file_list error.'    

In [3]:
def dcm2jpg_array(array):
    return cv2.normalize(array, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
     

In [42]:
def deface_features(dcm_list, result_path, result_png_path, features):
    #features
    # 0 : eye, 1 : nose, 2 : ear, 3 : mouth
    for dcm_path in dcm_list:
        folder = result_path+str(dcm_path.split('/')[-1].split('.')[0])
        folder_png = result_png_path+str(dcm_path.split('/')[-1].split('.')[0])
        file_name = str(dcm_path.split('/')[-1])

        if not os.path.exists(folder):
            os.makedirs(folder)    
        if not os.path.exists(folder_png):
            os.makedirs(folder_png)           
            
        ds = pydicom.dcmread(dcm_path)
        array = ds.pixel_array
        array = dcm2jpg_array(array)
        array = Image.fromarray(array)
        x = np.array(array)
        results = model.predict(array, conf=0.5)
        for result in results:
            boxes = result.boxes.cpu().numpy()
            for box in boxes:
                if box.cls in features:
                    r = box.xyxy[0].astype(int)
                    x = cv2.rectangle(x, r[:2], r[2:], (0, 0, 0), -1) 
        png = Image.fromarray(x)
        png.save(folder_png+'\\'+file_name+'.png')

        img_gray = x.astype("uint16") * 16
        arr = img_gray
        ds.compress(RLELossless, arr)
        ds.save_as(folder+'\\'+file_name+'.dcm')

    
    

In [44]:
dcmfile = './sample-head-CT.dcm'
deface_features([dcmfile], './', './', [0,1,2,3])

0: 640x640 1 nose, 1 ear, 485.6ms
Speed: 1.0ms preprocess, 485.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
